# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "12g")
         .config("spark.executor.cores", "2")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
         .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
         .getOrCreate())


sc = spark.sparkContext
sc.addFile('/home/jovyan/Work/ej/paquetes/nlppen/nlppen.zip')
sc.uiWebUrl

21/11/18 20:52:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/18 20:52:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


'http://69abb479a5e1:4041'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:

anno_inf = 2010
anno_sup = 2021

terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}


seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales2', filtro=f'anno >= {anno_inf} AND anno < {anno_sup}')
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True, partition_cols=['anno'], precargar=False)

estructurales = SentenciasEstructurales(seleccion)

seleccion.sdf.count()

21/11/18 20:52:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


151269

In [ ]:
spark.read.parquet('../../src/datasets/complete').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').count()

# Formar dataset de sentencias estructurales

In [ ]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)

columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

In [ ]:
from pyspark.sql.types import *
columnas = {
    'inst internacionales' : ArrayType(StringType())
}

estructurales.extraerInstrumentosInternacionales(columnas, True)

columnas = {
    'derechos Norm' : ArrayType(StringType()),
    'derechos GenXPat' : ArrayType(StringType())
}
estructurales.extraerDerechos(columnas, True)

columnas = {
    'derechos Acotados' : ArrayType(StringType()),
    'derechos General' : ArrayType(StringType()),
    'derechos Fundamental' : ArrayType(StringType()),
    'derechos Humano' : ArrayType(StringType())
}

estructurales.extraerDerechosSinNormalizar(columnas, True)

columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
estructurales.agregarIDSentencia(columnas, sentencias, True)

columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()),
        'citasIDVoto' : MapType(StringType(), StringType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas

In [ ]:
estructurales.seleccion.guardarDatos(parquet_file='extracciones2', partition_cols=['anno'], borrar=False)

In [4]:
estructurales.seleccion.sdf.limit(10).toPandas().columns

Index(['archivo', 'txt', 'cleanText', 'num', 'fechahora_ext', 'redactor_ext',
       'tipoResolucion_ext', 'termino_ext', 'voto_salvado', 'tema',
       'Recurrente_ents', 'Cantón_ents', 'Fecha_ents', 'Constitución_ents',
       'Redactor_ents', 'Ley_ents', 'EntidadPublica_ents', 'tema_prob',
       'termino_lst', 'expediente', 'Fechahora_ents', 'CitaSentencias_ents',
       'Recurrido_ents', 'Reglamento_ents', 'Magistrado_ents', '_id', 'anno',
       'seguimiento', 'se_ordena', 'plan', 'plazo'],
      dtype='object')

In [ ]:
estructurales.seleccion.sdf.dropNull('FechaSolicitud').groupby('anno').count()

In [ ]:
estructurales.seleccion.sdf.select('FechaSolicitud', 'anno').filter('FechaSolicitud is not Null').toPandas().count()

In [ ]:
estructurales.seleccion.sdf.limit(15).toPandas()

# Cargar procesamiento de sentencias estructurales.

In [6]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./datasets', datasets_path='./datasets/estructurales2')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
seleccion.sdf.columns

Py4JJavaError: An error occurred while calling o113.parquet.
: java.lang.AssertionError: assertion failed: Conflicting directory structures detected. Suspicious paths:
	file:/home/jovyan/work/ej/paquetes/nlppen/datasets/estructurales/terminos.parquet_prev
	file:/home/jovyan/work/ej/paquetes/nlppen/datasets/estructurales/extracciones2
	file:/home/jovyan/work/ej/paquetes/nlppen/datasets/estructurales/terminos.parquet
	file:/home/jovyan/work/ej/paquetes/nlppen/datasets/estructurales2/terminos.parquet

If provided paths are partition directories, please set "basePath" in the options of the data source to specify the root directory of the table. If there are multiple root directories, please load them separately and then union them.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:172)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:1

In [ ]:
from pyspark.sql.types import *
columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

# Zona de trabajo (ignorar)

In [ ]:
s = estructurales.seleccion.sdf
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'se_ordena_PER'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'plazosDefinidos'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1])